In [1]:
import torch

In [2]:
import torch.nn as nn

In [3]:
import importlib

In [4]:
import tasks

In [5]:
task = 'normals'

In [6]:
target_task = importlib.import_module(f'tasks.{task}')

In [7]:
target_task.cfg['eval_dir']

'eval/normals'

In [8]:
target_task.cfg['weights_file']

'weights/normals.pth'

In [9]:
cfg_loader = target_task.CfgLoader()

In [10]:
cfg = cfg_loader.get_cfg('cuda')

In [11]:
image_dir = cfg['image_dir']

In [12]:
dataset = cfg['dataset']

In [13]:
img, target = dataset[0]

In [14]:
model = cfg['model']

In [15]:
weights_file = cfg['weights_file']

In [16]:
model.load_state_dict(torch.load(f'./{weights_file}'))

In [17]:
model.eval()

EncoderDecoder(
  (encoder): Resnet11Encoder128x128(
    (resnet): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1)

In [18]:
from torch.autograd import Variable

In [19]:
output = model(img.view(1,3,128,128).cuda())

In [20]:
output.shape

torch.Size([1, 3, 128, 128])

In [21]:
output.shape[3]

128

In [22]:
# output [B, C, H, W]

In [23]:
conv0 = nn.Conv2d(3, 64, kernel_size = 4, stride = 2, padding = 1)

In [24]:
x = conv0(output.cpu())

In [25]:
relu0 = nn.LeakyReLU(0.2, inplace=True)

In [26]:
x = relu0(x)

In [27]:
x.shape

torch.Size([1, 64, 64, 64])

In [28]:
#import matplotlib.pyplot as plt

In [29]:
#import matplotlib.image as mpimg

In [30]:
#import numpy as np

In [31]:
#to_img = (lambda x: (0.5*(x+1)).clamp(0.0,1.0))

In [32]:
conv1 = nn.Conv2d(64, 128, kernel_size = 4, stride = 2, padding = 1)

In [33]:
relu1 = nn.LeakyReLU(0.2, inplace=True)

In [34]:
x = conv1(x)

In [35]:
bn0 = nn.BatchNorm2d(128)

In [36]:
 x = relu1(x)

In [37]:
x=bn0(x)

In [38]:
x.shape

torch.Size([1, 128, 32, 32])

In [39]:
conv2 = nn.Conv2d(128,256, kernel_size = 4, stride = 2, padding = 1)

In [40]:
x = conv2(x)

In [41]:
relu2 = nn.LeakyReLU(0.2, inplace=True)

In [42]:
bn1 = nn.BatchNorm2d(256)

In [43]:
x=relu2(x)

In [44]:
x=bn1(x)

In [45]:
x.shape

torch.Size([1, 256, 16, 16])

In [46]:
conv3 = nn.Conv2d(256,512, kernel_size = 4, stride = 2, padding = 1)

In [47]:
bn2 = nn.BatchNorm2d(512)

In [48]:
relu3 = nn.LeakyReLU(0.2, inplace=True)

In [49]:
x=relu3(bn2(conv3(x)))

In [50]:
x.shape

torch.Size([1, 512, 8, 8])

In [51]:
pad0=nn.ZeroPad2d((1,0,1,0))

In [52]:
x=pad0(x)

In [53]:
conv4 =  nn.Conv2d(512, 512, 4, padding=1)

In [54]:
bn3=nn.BatchNorm2d(512)

In [55]:
relu4 = nn.LeakyReLU(0.2, inplace=True)

In [56]:
x=relu4(bn3(conv4(x)))

In [57]:
x.shape

torch.Size([1, 512, 8, 8])

In [58]:
pad1=nn.ZeroPad2d((1,0,1,0))

In [59]:
x=pad1(x)

In [60]:
conv5 =  nn.Conv2d(512, 1, 4, padding=1,bias=False)

In [61]:
x=conv5(x)

In [62]:
x.shape

torch.Size([1, 1, 8, 8])

In [63]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels*2, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1, bias=False)
        )

    def forward(self, img_A, img_B):
        # Concatenate image and condition image by channels to produce input
        img_input = torch.cat((img_A, img_B), 1)
        return self.model(img_input)

In [64]:
d = Discriminator(3)

In [65]:
output.shape

torch.Size([1, 3, 128, 128])

In [66]:
target.shape

torch.Size([3, 128, 128])

In [67]:
z=d(output.cpu(), target.view(1,3,128,128))

In [68]:
z.shape

torch.Size([1, 1, 8, 8])

In [69]:
sig0 = torch.nn.Sigmoid()

In [70]:
x = sig0(x)

In [71]:
a0 = nn.AvgPool2d(kernel_size=8)

In [72]:
x=a0(x)

In [73]:
x.shape

torch.Size([1, 1, 1, 1])

In [74]:
x

tensor([[[[0.4690]]]], grad_fn=<AvgPool2DBackward>)

In [75]:
import models as r

In [76]:
d_test = r.Discriminator()

In [77]:
d_out = d_test(output.cpu(), target.view(1,3,128,128))

In [78]:
d_out.shape

torch.Size([1, 1, 1, 1])

In [79]:
d_out

tensor([[[[0.4951]]]], grad_fn=<AvgPool2DBackward>)

In [80]:
import numpy as np

In [82]:
model_parameters = filter(lambda p: p.requires_grad, d_test.parameters())

In [83]:
params = sum([np.prod(p.size()) for p in model_parameters])

In [84]:
print(params)

6965440


In [85]:
print((params*4)/(1024*1024))

26.571044921875


In [87]:
print(sum(p.numel() for p in d_test.parameters() if p.requires_grad))

6965440
